In [3]:
import findspark
findspark.init('C:\\Spark')


In [4]:
import sys

In [5]:
from pyspark import SparkConf , SparkContext
from math import sqrt

In [6]:
def DuplicateFilter(UserRatings):  #def de fonction ou on va eliminer la redondance 
    ratings=UserRatings[1]
    (movie1,rating1)=ratings[0]
    (movie2,rating2)=ratings[1]
    return movie1 < movie2
    

In [12]:
def loadMoviesNames() : #bch naamlou def lil structure du dictionnaire disant n7adhrou lil variable Broadcast
    moviesNames ={}   #khtr wala aandna tuple(dictionnaire)
    with open("C:/Users/meher/Desktop/python/dataset/ml-100k/u.ITEM") as f:  #with open elle va creer un dictionnaire qui existe deja dans le rep
        for line in f:    #parcequ'on va parcourir le fichier u.item et on va prendre les tires des films lihoma item[1] 
            fields=line.split('|')
            moviesNames[int(fields[0])]=fields[1]
    return moviesNames       

In [19]:
def PairsFilms(UserRatings):
    ratings=UserRatings[1]
    (movie1,rating1)=ratings[0]
    (movie2,rating2)=ratings[1]
    return ((movie1,movie2),(rating1,rating2))
    
    

In [8]:
def computeCosSimilarity (ratingsPairs) :
    num_pairs=0
    XX=XY=YY=0
    for ratingX , ratingY in ratingsPairs :
        XX=XX+ratingX * ratingX
        XY=XY+ratingX * ratingY
        YY=YY+ratingY * ratingY
        num_pairs+=1
    Denominateur=sqrt(XX)+sqrt(YY)
    final=0
    if(Denominateur):
        final=XY / (float(Denominateur))
    return (final,num_pairs)

In [9]:
conf = SparkConf().setMaster ("local[*]").setAppName("Similar_film")
sc = SparkContext (conf = conf)

In [10]:
lines = sc.textFile("C:/Users/meher/Desktop/python/dataset/ml-100k/u.data")

In [13]:
nameDict=loadMoviesNames()

In [14]:
ratings =lines.map(lambda x:x.split()).map(lambda y :(int(y[0]),(int(y[1]),float(y[2])))) #l map lawla aamalna beha ta9ssim % l split w thenya aamalna structure mtaa (userID,(FilmID,Rating)) li homa (key,value)

In [15]:
joinedRatings=ratings.join(ratings)

In [16]:
UniqueRatings=joinedRatings.filter(DuplicateFilter) 
    

In [20]:
moviePairs=UniqueRatings.map(PairsFilms)

In [21]:
movisPairsRatings=moviePairs.groupByKey()

In [22]:
moviesPairsSim=movisPairsRatings.mapValues(computeCosSimilarity).cache() #pour garder cette valeur dans la RAM

In [26]:
IDFilm=17

In [27]:
resultsFiltred = moviesPairsSim.filter(lambda Sim: (Sim[0][0]==IDFilm or Sim[0][1]==IDFilm))

In [28]:
resultsF=resultsFiltred.map(lambda res:(res[1],res[0])).sortByKey(ascending=False).take(5)  #hni glebna l key wl value kenet ((idFilm1,idfilm2),(score,nbre)) (key,value) wallet aandi l aaks ((score,nbre),(idFilm1,idfilm2)) aala khtr bch naaml tri mtaa l score w hni aandna kn sortedbykey

In [29]:
print ("Top 5 films similaires à " + nameDict[IDFilm])
for result in resultsF :
    (Similarity , Films) = result
    FilmSimilaire= Films[0]
    if (FilmSimilaire==IDFilm):
        FilmSimilaire= Films[1]
        print(nameDict[FilmSimilaire]+":\t le score est :"+str(Similarity[0])+"\t Nombre d'occurence :" +str(Similarity[1]))

Top 5 films similaires à From Dusk Till Dawn (1996)
Pulp Fiction (1994):	 le score est :16.662164035141572	 Nombre d'occurence :86
Star Wars (1977):	 le score est :16.32816140967038	 Nombre d'occurence :84
Raiders of the Lost Ark (1981):	 le score est :15.887627125172587	 Nombre d'occurence :81
Silence of the Lambs, The (1991):	 le score est :15.452030210112612	 Nombre d'occurence :76
Empire Strikes Back, The (1980):	 le score est :15.435101693247365	 Nombre d'occurence :76
